## 代码要求：使用五倍交叉，训练100轮，使用早停技术

# test

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from torch.autograd import Variable
import torch.nn as nn
from torchvision import datasets,transforms

In [ ]:
import torch
import pandas as pd
import numpy as np
import skimage
from torch.utils.data import Dataset, DataLoader
torch.manual_seed(1)  # reproducible
torch.set_default_tensor_type(torch.DoubleTensor)
from PIL import Image
transform = transforms.Compose([
    transforms.ToTensor(),  # 将图片转换为Tensor,归一化至[0,1]
])


In [ ]:
# dataset=MyDataset('/content/drive/MyDrive/100/test1/train.npy',"/content/drive/MyDrive/100/test1/label.npy")
# # dataset_test=MyDataset('/content/drive/MyDrive/100/test1/x_test.npy',"/content/drive/MyDrive/100/test1/y_test.npy")
# train_loader= DataLoader(dataset, batch_size=32, shuffle=True, pin_memory=True)
# # test_loader= DataLoader(dataset_test, batch_size=32, shuffle=False, pin_memory=True)

In [ ]:
# for data, target in test_loader:
#     print(data.shape)
#     print("DDDDDDDDD")
#     print(target)
#     break

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=5,padding=2)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=5)
        self.mp = nn.MaxPool2d(3)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(9216,640)
        self.fc2 = nn.Linear(640,64)
        self.fc3 = nn.Linear(64,8)
        self.logsoftmax = nn.LogSoftmax(dim=1)
        
    def forward(self,x):
        in_size = x.size(0)
        out = self.relu(self.mp(self.conv1(x)))
        out = self.relu(self.mp(self.conv2(out)))
        out = out.view(in_size, -1)
        out = self.relu(self.fc1(out))
        out = self.relu(self.fc2(out))
        out = self.fc3(out)
        return self.logsoftmax(out)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cuda', index=0)

In [ ]:
model = Net().to(device)

In [ ]:
model

Net(
  (conv1): Conv2d(1, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1))
  (mp): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (fc1): Linear(in_features=9216, out_features=640, bias=True)
  (fc2): Linear(in_features=640, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=8, bias=True)
  (logsoftmax): LogSoftmax(dim=1)
)

In [ ]:
loss_fn = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.0015,momentum=0.5)

In [ ]:
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold as kFold

In [ ]:
x_train=np.load("/content/drive/MyDrive/summer_start/data_generate/train_1channel.npy")
y_train=np.load("/content/drive/MyDrive/summer_start/data_generate/label.npy")

In [ ]:
kfold =kFold(n_splits=5,shuffle=True,random_state=1)

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data,label):
        self.data = data #加载npy数据
        self.label = label
        self.transforms = transform #转为tensor形式
    def __getitem__(self, index):
        hdct= self.data[index, :, :, :]  # 读取每一个npy的数据
        hdct = np.squeeze(hdct)  # 删掉一维的数据，就是把通道数这个维度删除
#         ldct = 2.5 * skimage.util.random_noise(hdct * (0.4 / 255), mode='poisson', seed=None) * 255 #加poisson噪声
#         hdct=Image.fromarray(np.uint8(hdct)) #转成image的形式
#         ldct=Image.fromarray(np.uint8(ldct)) #转成image的形式
        hdct= self.transforms(hdct)  #转为tensor形式
#         ldct= self.transforms(ldct)  #转为tensor形式
        return hdct, self.label[index] #返回数据还有标签
    def __len__(self):
        return self.data.shape[0] #返回数据的总个数

In [ ]:
model

Net(
  (conv1): Conv2d(1, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1))
  (mp): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (fc1): Linear(in_features=9216, out_features=640, bias=True)
  (fc2): Linear(in_features=640, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=8, bias=True)
  (logsoftmax): LogSoftmax(dim=1)
)

In [ ]:
x_train.shape

(13887, 1, 236, 236)

In [140]:
labels=[0,1,2,3,4,5,6,7]
list_score=[]
for fold, (train_index, test_index) in enumerate(kfold.split(x_train, y_train)):
    print("train_index",train_index)
    print("test_index",test_index)
    ### Dividing data into folds
    x_train_fold = x_train[train_index]
    x_test_fold = x_train[test_index]
    y_train_fold = y_train[train_index]
    y_test_fold = y_train[test_index]
    train = MyDataset(x_train_fold, y_train_fold)
    test = MyDataset(x_test_fold, y_test_fold)
    train_loader = DataLoader(train, batch_size = 8, shuffle = True)
    test_loader = DataLoader(test, batch_size = 8, shuffle = False)
    print("fold:",fold)
    max_score=[]
    # model = Net().to(device)
    best_acc=0
    model = Net().to(device)
    loss_fn = nn.NLLLoss()
    optimizer = torch.optim.SGD(model.parameters(),lr=0.001,momentum=0.5)
    for i in range(50):
      list1=[]
      for t, (data, target) in enumerate(train_loader):
        # print("t",t)
        data,target = Variable(data.to(device)),Variable(target.to(device))
        # print(data.shape)
        pred = model(data.double().to(device))
        loss = loss_fn(pred,target)
        
        list1.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print("loss",loss.item())
      print("netloss",i,np.mean(list1))
      with torch.no_grad():
        list_loss=[]
        correct = 0
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data.double().to(device))
            # get the index of the max log-probability
            pred = output.data.max(1, keepdim=True)[1]
            pred_loss=np.exp(output.data.cpu())
            logsloss=log_loss(target.cpu().detach().numpy(),pred_loss,labels=labels)
            list_loss.append(logsloss)
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        print('accuracy:{:.3f}%'.format(
            100. * correct / len(test_loader.dataset)))
        val_acc=correct / len(test_loader.dataset)
        print("log_loss",sum(list_loss)/len(list_loss))
        print("*"*20)
        if val_acc > best_acc:
            best_acc = val_acc
            es = 0
        else:
            es += 1
            print("Counter {} of 5".format(es))
            if es > 5:
                print("Early stopping with best_acc: ", best_acc, "and val_acc for this epoch: ", val_acc, "...")
                break
    list_score.append(best_acc)
    print("*"*40)
from numpy import mean
print(mean(list_score))


train_index [    0     1     2 ... 13884 13885 13886]
test_index [    4     5     6 ... 13861 13872 13877]
fold: 0
netloss 0 1.574258468085366
accuracy:57.163%
log_loss 1.273316631583682
********************
netloss 1 1.2809590645847864
accuracy:59.539%
log_loss 1.1920845880889246
********************
netloss 2 1.1521897993485806
accuracy:61.591%
log_loss 1.1096441158206727
********************
netloss 3 1.093414050964599
accuracy:63.031%
log_loss 1.0664146089176103
********************
netloss 4 1.0834854955750568
accuracy:64.147%
log_loss 1.053935777932527
********************
netloss 5 0.9802543300001002
accuracy:60.943%
log_loss 1.0335904870332167
********************
Counter 1 of 5
netloss 6 0.9128123831520891
accuracy:68.251%
log_loss 0.9300401835331011
********************
netloss 7 0.8739704373137829
accuracy:70.194%
log_loss 0.9357006668089364
********************
netloss 8 0.9793489500529547
accuracy:67.855%
log_loss 1.0233554850724094
********************
Counter 1 of 5
netl

In [141]:
list_score

[tensor(0.8297),
 tensor(0.8287),
 tensor(0.8455),
 tensor(0.8192),
 tensor(0.8063)]

In [142]:
print(mean(list_score))

0.8258798424682021
